In [1]:
import torch
import torchvision
import os
import torch.nn as nn
from ddpm.diffusion_model import DiffusionModel

import ddpm.config as _config
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from ddpm.data import CIFAR10_CLASSES

In [2]:
_config.DEBUG = False

cpt_path should be the path to the checkpoint file

there is bug in torch.load, sometimes it can load successfully, sometimes it can't. Even we don't change anything,

In [3]:
cpt_path = 'epoch_24.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
cpt = torch.load(cpt_path, map_location=device, weights_only=False)
model = DiffusionModel(cpt['config'])
model.load_state_dict(cpt['model'])
model = nn.DataParallel(model).to(device)

model.eval()
print("model loaded")

# sample

In [5]:
save_dir = 'cifar10_samples'

sample image_num images each label

In [ ]:
# image_num = 9
# for label in range(10):
#     print(CIFAR10_CLASSES[label])
#     labels = torch.ones(image_num, dtype=torch.long, device=device) * label
#     samples = model.module.sample(shape=(image_num, 3, 32, 32), device=device, y=labels)

#     for i, sample in enumerate(samples):
#         processed_sample = ((sample + 1) / 2).clip(0, 1)
#         filename = f"{save_dir}/{CIFAR10_CLASSES[label]}_{i}.png"

#         torchvision.utils.save_image(processed_sample, filename)
#         print(f"Saved {filename}")

many images with random labels in one image file

In [ ]:
## mixed label

# image_num = 20 * 36
# import random

# # Generate random labels between 0 and 9
# labels = torch.tensor([random.randint(0, 9) for _ in range(image_num)], dtype=torch.long, device=device)

# # Sample images from the model
# samples = model.module.sample(shape=(image_num, 3, 32, 32), device=device, y=labels)
# processed_samples = [((sample + 1) / 2).clip(0, 1) for sample in samples]

# # Create a figure to hold the images
# fig, axes = plt.subplots(20, 36, figsize=(36, 20))

# for i, sample in enumerate(processed_samples):
#     row = i // 36
#     col = i % 36
#     ax = axes[row, col]
#     ax.imshow(sample.permute(1, 2, 0).cpu().numpy())
#     ax.axis('off')

# # Save the figure
# plt.tight_layout()
# plt.savefig(f'{save_dir}/random_label_samples.png')
# plt.show()



image_num images each label in one image file

In [ ]:
## 

# image_num = 14
# save_dir = 'cifar10_samples'
# for label in range(10):
#     print(CIFAR10_CLASSES[label])
#     labels = torch.ones(image_num, dtype=torch.long, device=device) * label
#     samples = model.module.sample(shape=(image_num, 3, 32, 32), device=device, y=labels)
#     processed_samples = [((sample + 1) / 2).clip(0, 1) for sample in samples]
    

#     # Create a figure to hold the images
#     fig, axes = plt.subplots(2, 7, figsize=(14, 4))
    
#     for i, sample in enumerate(processed_samples):
#         row = i // 7
#         col = i % 7
#         ax = axes[row, col]
#         ax.imshow(sample.permute(1, 2, 0).cpu().numpy())
#         ax.axis('off')
    
#     # Save the figure
#     plt.tight_layout()
#     plt.savefig(f'{save_dir}/{CIFAR10_CLASSES[label]}_samples.png')
#     plt.show()

# denoising progress

In [ ]:
save_steps = [i * 30 for i in range(20)] # steps you want to save
labels = torch.ones(16, dtype=torch.long, device=device) * 1 # Automobile
samples = model.module.sample_intermediate(shape=(16, 3, 32, 32), device=device, y=labels, save_steps=save_steps)

processed_samples = [((sample + 1) / 2).clip(0, 1) for sample in samples]
num_steps = len(processed_samples)
num_samples = processed_samples[0].size(0)

plt.figure(figsize=(num_steps * 2, num_samples * 2))

for i in range(num_samples):
    for j in range(num_steps):
        plt.subplot(num_samples, num_steps, i * num_steps + j + 1)
        plt.imshow(processed_samples[j][i].permute(1, 2, 0).cpu().numpy())
        plt.axis('off')
        # if i == 0:  # 只在第一行添加标题
        #     plt.title(f'Step {save_steps[j]}')

plt.tight_layout()
plt.savefig('denoising_process.png')
plt.show()